## 0. Setup

In [1]:
# Import dependencies
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
import PIL
PIL.PILLOW_VERSION = PIL.__version__
from torchvision import datasets, transforms
import numpy as np
import random

In [2]:
# Set up your device 
cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if cuda else "cpu")

In [3]:
# Set up random seed to 1008. Do not change the random seed.
# Yes, these are all necessary when you run experiments!
seed = 1008
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if cuda:
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

## 1. Data: MNIST
#### Load the MNIST training and test dataset using $\texttt{torch.utils.data.DataLoader}$ and $\texttt{torchvision.datasets}$. 

Hint: You might find Alf's notebook useful: https://github.com/Atcold/pytorch-Deep-Learning/blob/master/06-convnet.ipynb, or see some of the PyTorch tutorials.

### 1.1. Load Training Set [4 pts]

In [4]:
# Load the MNIST training set with batch size 128, apply data shuffling and normalization
# train_loader = TODO

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=128, shuffle=True)

### 1.1. Load Test Set [4 pts]

In [5]:
# Load the MNIST test set with batch size 128, apply data shuffling and normalization
# test_loader = TODO

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=128, shuffle=True)

## 2. Models
#### You are going to define two convolutional neural networks which are trained to classify MNIST digits

### 2.1. CNN without Batch Norm [5 pts]

In [6]:
# Fill in the values below that make this network valid for MNIST data

conv1_in_ch = 1
conv2_in_ch = 20
fc1_in_features = 50*4*4
fc2_in_features = 500
n_classes = 10

In [7]:
class NetWithoutBatchNorm(nn.Module):
    def __init__(self):
        super(NetWithoutBatchNorm, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=conv1_in_ch, out_channels=20, kernel_size=5, stride=1)
        self.conv2 = nn.Conv2d(in_channels=conv2_in_ch, out_channels=50, kernel_size=5, stride=1)
        self.fc1 = nn.Linear(in_features=fc1_in_features, out_features=500)
        self.fc2 = nn.Linear(in_features=fc2_in_features, out_features=n_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = x.view(-1, fc1_in_features) # reshaping
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        # Return the log_softmax of x.
        x = F.log_softmax(x, dim=1)
        return x

### 2.2. CNN with Batch Norm [5 pts]

In [8]:
# Fill in the values below that make this network valid for MNIST data

conv1_bn_size = 20
conv2_bn_size = 50
fc1_bn_size = 500

In [9]:
# Define the CNN with architecture explained in Part 2.2
class NetWithBatchNorm(nn.Module):
    def __init__(self):
        super(NetWithBatchNorm, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=conv1_in_ch, out_channels=20, kernel_size=5, stride=1)
        self.conv1_bn = nn.BatchNorm2d(conv1_bn_size)
        self.conv2 = nn.Conv2d(in_channels=conv2_in_ch, out_channels=50, kernel_size=5, stride=1)
        self.conv2_bn = nn.BatchNorm2d(conv2_bn_size)
        self.fc1 = nn.Linear(in_features=fc1_in_features, out_features=500)
        self.fc1_bn = nn.BatchNorm1d(fc1_bn_size)
        self.fc2 = nn.Linear(in_features=fc2_in_features, out_features=n_classes)

    def forward(self, x):
        x = F.relu(self.conv1_bn(self.conv1(x)))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = F.relu(self.conv2_bn(self.conv2(x)))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = x.view(-1, fc1_in_features)
        x = F.relu(self.fc1_bn(self.fc1(x)))
        x = self.fc2(x)
        # Return the log_softmax of x.
        x = F.log_softmax(x, dim=1)
        return x

## 3. Training & Evaluation

### 3.1. Define training method [10 pts]

In [10]:
def train(model, device, train_loader, optimizer, epoch, log_interval = 100):
    # Set model to training mode
    model.train()
    # Loop through data points
    for batch_idx, (data, target) in enumerate(train_loader):
        #pass # remove once implemented
    
        # Send data and target to device
        data, target = data.to(device), target.to(device)
        
        # Zero out the ortimizer
        optimizer.zero_grad()
        
        # Pass data through model
        output = model(data)
        
        # Compute the negative log likelihood loss
        loss = F.nll_loss(output, target)
        
        # Backpropagate loss
        loss.backward()
        
        # Make a step with the optimizer
        optimizer.step()
        
        # Print loss (uncomment lines below once implemented)
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

### 3.2. Define test method [10 pts]

In [11]:
accuracy_list = []

# Define test method
def test(model, device, test_loader):
    # Set model to evaluation mode
    model.eval()
    # Variable for the total loss 
    test_loss = 0
    # Counter for the correct predictions
    num_correct = 0
    
    # don't need autograd for eval
    with torch.no_grad():
        # Loop through data points
        for data, target in test_loader:
            # pass # remove once implemented
        
            # Send data to device
            data, target = data.to(device), target.to(device)
            
            # Pass data through model
            output = model(data)
            
            # Compute the negative log likelihood loss with reduction='sum' and add to total test_loss
            test_loss += F.nll_loss(output, target, reduction='sum').item()
            
            # Get predictions from the model for each data point
            pred = output.data.max(1, keepdim=True)[1]
            
            # Add number of correct predictions to total num_correct 
            num_correct += pred.eq(target.data.view_as(pred)).cpu().sum().item()
    
    # Compute the average test_loss
    test_loss /= len(test_loader.dataset)
    accuracy = 100. * num_correct / len(test_loader.dataset)
    accuracy_list.append(accuracy)
    
    # Print loss (uncomment lines below once implemented)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, num_correct, len(test_loader.dataset),
        100. * num_correct / len(test_loader.dataset)))

### 3.3 Train NetWithoutBatchNorm() [5 pts]

In [12]:
# Deifne model and sent to device
model = NetWithoutBatchNorm()
model.to(device)

# Optimizer: SGD with learning rate of 1e-2 and momentum of 0.5
optimizer = optim.SGD(model.parameters(), lr = 0.01, momentum = 0.5)

# Training loop with 10 epochs
for epoch in range(1, 10 + 1):
    # pass # remove once implemented

    # Train model
    train(model, device, train_loader, optimizer, epoch)

    
    # Test model
    test(model, device, test_loader)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.312853
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.508909
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.338842
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.321634
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.223016

Test set: Average loss: 0.1599, Accuracy: 9521/10000 (95%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.232539
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.103974
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.121136
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.177091
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.111958

Test set: Average loss: 0.0939, Accuracy: 9726/10000 (97%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.087348
Train Epoch: 3 [12800/60000 (21%)]	Loss: 0.054673
Train Epoch: 3 [25600/60000 (43%)]	Loss: 0.100732
Train Epoch: 3 [38400/60000 (64%)]	Loss: 0.060889
Train Epoch: 3 [51200/60000 (85%)]	Loss: 0.062613

Test set: Average loss: 0.0809, Accuracy: 9751/10000 (98%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 0.026950
Train Epoch: 4 [12800/60000 (21%)]	Lo

### 3.4 Train NetWithBatchNorm() [5 pts]

In [13]:
# Deifne model and sent to device
model = NetWithBatchNorm()
model.to(device)

# Optimizer: SGD with learning rate of 1e-2 and momentum of 0.5
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum = 0.5)

# Training loop with 10 epochs
for epoch in range(1, 10 + 1):
    # pass # remove once implemented
    
    # Train model
    train(model, device, train_loader, optimizer, epoch)
    
    # Test model
    test(model, device, test_loader)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.384663
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.253188
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.179382
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.176117
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.101649

Test set: Average loss: 0.0844, Accuracy: 9788/10000 (98%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.062142
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.034514
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.070800
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.070866
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.083214

Test set: Average loss: 0.0530, Accuracy: 9863/10000 (99%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.040267
Train Epoch: 3 [12800/60000 (21%)]	Loss: 0.069403
Train Epoch: 3 [25600/60000 (43%)]	Loss: 0.073169
Train Epoch: 3 [38400/60000 (64%)]	Loss: 0.025606
Train Epoch: 3 [51200/60000 (85%)]	Loss: 0.047004

Test set: Average loss: 0.0409, Accuracy: 9888/10000 (99%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 0.079442
Train Epoch: 4 [12800/60000 (21%)]	Lo

## 4. Empirically, which of the models achieves higher accuracy faster? [2 pts]

Answer: The model using Batch Normalization achieves accuracy faster than the model without. 